# Import and pre-definitions

In [1]:
import os
MAIN_PATH = r'/home/luis-felipe'
DATA_PATH = os.path.join(MAIN_PATH,'data')
PATH_MODELS = os.path.join(MAIN_PATH,'torch_models')
FIGS_PATH = os.path.join(MAIN_PATH,'results','figs')

In [2]:
import torch
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.set_default_dtype(torch.float64)
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

True


In [4]:
import sys
sys.path.insert(1, '..')
sys.path.insert(1, '../..')

import models
from utils import measures,metrics
from data_utils import upload_logits,split_data
import post_hoc
from post_hoc import other_methods,optimize_other_methods

# Evaluate logits

In [5]:
DATASET = 'ImageNet'
VAL_SIZE = 0.1 #5000
SUB_VAL_SIZE = 0.2
METRIC = metrics.AURC
NUM_SPLITS = 10

In [6]:
methods = {'BK': other_methods.BK,
          'ETS': other_methods.ETS,
          'HTS':other_methods.HTS}
optimizers = {'BK': optimize_other_methods.BK,
          'ETS': optimize_other_methods.ETS,
          'HTS': optimize_other_methods.HTS}

In [7]:
from collections import defaultdict
d_naurc = {m:defaultdict(list) for m in methods.keys()}
d_baseline = defaultdict(list)
acc = defaultdict(list)

In [8]:
seed = SEED
for i in range(NUM_SPLITS):
    print(i+1)
    for model_arc in models.list_models():
        print(model_arc)
        #if model_arc in df['model'].values:
        #    print('ja foi')
        #    continue
        if 'quantized' in model_arc:
            print('quantized - skip')
            continue
        with torch.no_grad():
            try : logits_val,labels_val,logits_test,labels_test = split_data.split_logits(*upload_logits(model_arc,DATASET,PATH_MODELS, 
                                split = 'test', device = dev),VAL_SIZE,seed = seed)
            except: 
                print('cant find')
                continue
        logits_val,labels_val = logits_val[:int(SUB_VAL_SIZE*labels_val.size(0))],labels_val[:int(SUB_VAL_SIZE*labels_val.size(0))]
        logits_val = post_hoc.centralize(logits_val)
        logits_test = post_hoc.centralize(logits_test)
        risk_val = measures.wrong_class(logits_val,labels_val).float()
        risk_test = measures.wrong_class(logits_test,labels_test).float()
        acc[model_arc].append((1-risk_test.mean().item()))

        for method,fn in methods.items():
            params = optimizers[method](logits_val,risk_val)
            d_naurc[method][model_arc].append(metrics.N_AURC(risk_test,fn(logits_test,*params)))

        d_baseline[model_arc].append(metrics.N_AURC(risk_test,measures.MSP(logits_test)))
    seed += 10
models_list = list(acc.keys())

1
alexnet
convnext_base
convnext_large
convnext_small
convnext_tiny
deeplabv3_mobilenet_v3_large
cant find
deeplabv3_resnet101
cant find
deeplabv3_resnet50
cant find
densenet121
densenet161
densenet169
densenet201
efficientnet_b0
efficientnet_b1
efficientnet_b2
efficientnet_b3
efficientnet_b4
efficientnet_b5
efficientnet_b6
efficientnet_b7
efficientnet_v2_l
efficientnet_v2_m
efficientnet_v2_s
fasterrcnn_mobilenet_v3_large_320_fpn
cant find
fasterrcnn_mobilenet_v3_large_fpn
cant find
fasterrcnn_resnet50_fpn
cant find
fasterrcnn_resnet50_fpn_v2
cant find
fcn_resnet101
cant find
fcn_resnet50
cant find
fcos_resnet50_fpn
cant find
googlenet
inception_v3
keypointrcnn_resnet50_fpn
cant find
lraspp_mobilenet_v3_large
cant find
maskrcnn_resnet50_fpn
cant find
maskrcnn_resnet50_fpn_v2
cant find
maxvit_t
mc3_18
cant find
mnasnet0_5
mnasnet0_75
mnasnet1_0
mnasnet1_3
mobilenet_v2
mobilenet_v3_large
mobilenet_v3_small
mvit_v1_b
cant find
mvit_v2_s
cant find
quantized_googlenet
quantized - skip
quant

In [9]:
baseline = np.array(list(d_baseline.values()))

In [10]:
import pickle

with open('extra_methods.pickle', 'wb') as handle:
    pickle.dump(d_naurc, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
for method in methods:
    d_t = d_naurc[method]
    string = f'{method}'
    string += f" & {post_hoc.significant(baseline - np.array(list(d_t.values())),0.01).mean():.5f}"+r' {\footnotesize $\pm$'+f"{post_hoc.significant(baseline - np.array(list(d_t.values())),0.01).mean(0).std():.5f}"+"}"
    print(string + r' \\')

BK & 0.03795 {\footnotesize $\pm$0.00067} \\
ETS & 0.05569 {\footnotesize $\pm$0.00165} \\
HTS & 0.05927 {\footnotesize $\pm$0.00280} \\


In [12]:
baseline = np.array(list(d_baseline.values()))
gain = {}
for method in methods:
    gain[method] = baseline-np.array(list(d_naurc[method].values()))


In [13]:
epsilon = 0.01
apg = {}
for method in methods:
    apg[method] = post_hoc.significant(gain[method],epsilon).mean()

In [14]:
apg

{'BK': 0.037948617271992446,
 'ETS': 0.05569025776923012,
 'HTS': 0.059268371673620125}

In [15]:

gain_p = {}
for method in d_naurc_p:
    gain_p[method] = baseline-np.array(list(d_naurc_p[method].values()))
epsilon = 0.01
apg_p = {}
for method in d_naurc_p:
    apg_p[method] = post_hoc.significant(gain_p[method],epsilon).mean()
apg_p

NameError: name 'd_naurc_p' is not defined